# Finalise twitter
Finalises the twitter data.
And persists it to s3.
See docs for more information.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.tag import export
from phoenix.tag import finalise
from phoenix.tag import object_filters

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parametrise the run execution date.
# Format of the run date
RUN_DATE_FORMAT = "%Y-%m-%d"
# This can be overwritten at execution time by Papermill to enable historic runs and backfills etc.
RUN_DATE = datetime.datetime.today().strftime(RUN_DATE_FORMAT)

YEAR_FILTER = 2021
MONTH_FILTER = 5
MONTH_PREFIX = f"year_filter={YEAR_FILTER}/month_filter={MONTH_FILTER}/"

# Set Artefacts URL
ARTIFACTS_BASE_URL = f"{artifacts.urls.get_local()}{RUN_DATE}/twitter/{MONTH_PREFIX}"

# OUTPUT
TWEETS_PERSIST = f"s3://buildup-dev-us-tables/tweets/parquet_exports/tweets_v1/{MONTH_PREFIX}{YEAR_FILTER}-{MONTH_FILTER}.parquet"

In [ ]:
# Display params.
print(
ARTIFACTS_BASE_URL,
TWEETS_PERSIST,
RUN_DATE,
sep='\n',
)

In [ ]:
# %env DASK_CLUSTER_IP=

In [ ]:
utils.dask_global_init()

In [ ]:
tweets_df = artifacts.dataframes.get(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "tweets_pulled")).dataframe

In [ ]:
tweets_df.head()

In [ ]:
objects = artifacts.dataframes.get(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "object_tensions")).dataframe

In [ ]:
objects.head()

In [ ]:
tweets_final = finalise.join_objects_to_tweets(objects, tweets_df)

In [ ]:
tweets_final.head()

In [ ]:
tweets_final.dtypes

In [ ]:
_ = artifacts.dataframes.persist(TWEETS_PERSIST, tweets_final)

In [ ]:
_ = artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "tweets_final"), tweets_final)

In [ ]:
artifacts.dataframes.read_schema(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "tweets_final"))